In [3]:
import numpy as np

In [4]:
# Define all the functions needed for mapping from parameters to the data space included defining the Q
#
# Some common inputs:
#   Obs_vec = np.array([o_1,o_2, $\ldots$, o_P])
#   Obs_err_std = standard deviation of observation error
#   S = number of repeated observations
#   lam_true = true parameter value
#   Data_vec = np.array([d_1, d_2, $\ldots$, d_S])

def Q_map(Obs_vec, Obs_err_std, S):
    return np.sqrt(S)/Obs_err_std * Obs_vec

def Noisy_data(lam_true, Obs_vec, Obs_err_std, S):
    return np.random.normal(size=(S,1))*Obs_err_std + np.dot(Obs_vec.transpose(), lam_true)

def b_repeated_obs(Data_vec, Obs_err_std, S):
    return -1/(np.sqrt(S)*Obs_err_std) * np.sum(Data_vec) #needs to be negative!!!! 

In [5]:
#Setup all things about the observation and map

Obs_vec_1 = np.array([[2], [1]])

Obs_vec_2 = np.array([[1], [1]])

Obs_err_std_1 = 0.25

Obs_err_std_2 = 0.2

S = 10

lam_true = np.array([[0.5],[1.5]])

print(np.dot(Obs_vec_1.transpose(),lam_true))

A_1 = Q_map(Obs_vec_1, Obs_err_std_1, S)

A_2 = Q_map(Obs_vec_2, Obs_err_std_2, S)

Data_vec_1 = Noisy_data(lam_true, Obs_vec_1, Obs_err_std_1, S)

Data_vec_2 = Noisy_data(lam_true, Obs_vec_2, Obs_err_std_2, S)

b_1 = b_repeated_obs(Data_vec_1, Obs_err_std_1, S)

b_2 = b_repeated_obs(Data_vec_2, Obs_err_std_2, S)

[[2.5]]


In [6]:
# Define the initial uncertainty parameters mu_init and sigma_init for an 
# initially assumed N(mu_init, sigma_init**2) distribution on $\Lambda$.
#
# NOTE: Using sigma_init for STANDARD DEVIATION and NOT covariance

lam_pert = np.array([[-0.2], [-0.1]]) #the perturbation from lam_true used to form an initial guess

mu_init = lam_true + lam_pert #the initial guess at lam_true

sigma_init = np.array([[0.75],[0.75]]) #the diagonal of the init standard deviation

Sigma_init = np.diag(sigma_init.flatten()) #the init standard deviation matrix

PF_cov_1 = np.dot(A_1.transpose(),np.dot(Sigma_init**2,A_1)) #the push-forward COVARIANCE

PF_cov_2 = np.dot(A_2.transpose(),np.dot(Sigma_init**2,A_2))

In [159]:
# Define the necessary functions to define the linear problem for determining the MUD point
#
# Some common inputs:

def gamma_vec(A, sigma_init, PF_cov):
    P = np.size(A)
    gamma = np.zeros((P,1))
    for i in range(P):
        gamma[i] = sigma_init[i]**2 * A[i] * (PF_cov-1)
    return gamma

def Alpha_mat(A, gamma, PF_cov):
    P = np.size(A)
    return np.dot(gamma, A.transpose()) + np.eye(P)*PF_cov

def Alpha_inv_SM(A, gamma, PF_cov):
    P = np.size(A)
    return 1/PF_cov*np.eye(P) - np.dot(gamma,A.transpose())/(PF_cov**2+PF_cov*np.dot(gamma.transpose(),A))

def beta_vec(mu_init, sigma_init, A, b, PF_cov):
    P = np.size(A)
    beta = np.zeros((P,1))
    for i in range(P):
        beta[i] = mu_init[i]*PF_cov-A[i]*sigma_init[i]**2*(np.dot(A.transpose(),mu_init)) - A[i]*sigma_init[i]**2*PF_cov*b
    return beta

In [160]:
print(mu_init)
print(A)
print(np.dot(A.transpose(),mu_init))

print(44.72*0.6+22.36*1.6)

[[0.3]
 [1.4]]
[[25.29822128]
 [12.64911064]]
[[25.29822128]]
62.608000000000004


In [161]:
gamma = gamma_vec(A,sigma_init,PF_cov)
Alpha = Alpha_mat(A, gamma, PF_cov)
print(Alpha)
Alpha_inv = Alpha_inv_SM(A, gamma, PF_cov)
print(Alpha_inv)
beta = beta_vec(mu_init, sigma_init, A, b, PF_cov)
print(beta)

[[162090.  80820.]
 [ 80820.  40860.]]
[[ 0.0004484  -0.00088691]
 [-0.00088691  0.00177877]]
[[201895.39607102]
 [101510.19803551]]


In [162]:
lam_MUD = np.dot(Alpha_inv, beta)
print(lam_MUD)
print()
print(np.dot(Obs_vec.transpose(),lam_MUD))
print()
print(np.dot(Obs_vec.transpose(),lam_true))

[[0.49812541]
 [1.49906271]]

[[2.49531353]]

[[2.5]]


In [163]:
def functional(mu_init, sigma_init, A, b, lam, PF_cov):
    return np.linalg.norm((lam-mu_init)/sigma_init)**2 + np.linalg.norm(np.dot(A.transpose(),lam)+b)**2 \
            - np.linalg.norm(np.dot(A.transpose(), lam-mu_init))**2/PF_cov

In [156]:
print(functional(mu_init,sigma_init, A, b, mu_init, PF_cov))
print()
print(functional(mu_init,sigma_init, A, b, lam_true, PF_cov))
print()
print(functional(mu_init,sigma_init, A, b, lam_MUD, PF_cov))
print()
print(functional(mu_init,sigma_init, A, b, lam_MUD+0.00001, PF_cov))

[[3893.50932816]]

[[0.71836105]]

[[-1.38777878e-17]]

[[1.44000402e-05]]
